In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pathlib
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
import shapely
import pprint
from pprint import pprint



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import utils.other
import utils.geo
import utils.coco
import utils.img


In [19]:
from utils.other import new_pickle, load_pickle, new_json, load_json

In [32]:
inpath_s2 = Path(r'/Users/chetnagupta/project-arms4ai/data/RGB_small.tif')
inpath_fields = Path(r'/Users/chetnagupta/project-arms4ai/data/marker2016_small.shp')

outpath = Path(r'/Users/chetnagupta/project-arms4ai/data/output/preprocessed')

# inpath_s2 = Path(r'data\RGB_small.tif')
# inpath_fields = Path(r'data\marker2016_small.shp')

# outpath = Path(r'output\preprocessed')


In [33]:
def prepare_vector(fp, out_crs, clipping_bounds):
    df = (gpd.read_file(str(fp), encoding='cp865')  # danish encoding
             .rename(columns={'Afgroede': 'lc_name', 'AfgKode': 'lc_id', 'JOURNALNUM': 'journalnr'})
             .drop(['GB', 'GEOMETRISK', 'MARKNUMMER'], axis=1)
             .pipe(utils.geo.explode_mp)
             .pipe(utils.geo.buffer_zero)
             .pipe(utils.geo.close_holes)
             .pipe(utils.geo.set_crs, 3044)
             .to_crs(out_crs)
             .pipe(utils.geo.clip, clip_poly=shapely.geometry.box(*clipping_bounds), explode_mp_=True)
             .pipe(utils.geo.reclassify_col, rcl_scheme=utils.geo.reclass_legend, 
                   col_classlabels='lc_name', col_classids='lc_id', drop_other_classes=True)
             .assign(geometry=lambda _df: _df.geometry.simplify(5, preserve_topology=True))
             .pipe(utils.geo.buffer_zero)
             .assign(area_sqm=lambda _df: _df.geometry.area)
             .pipe(utils.geo.reduce_precision, precision=4)
             .reset_index(drop=True)
             .assign(fid=lambda _df: range(0, len(_df.index)))
             .filter(['journalnr', 'lc_id', 'lc_name', 'r_lc_id', 'r_lc_name', 'area_sqm', 'fid', 'geometry']))
    return df

outpath_fields = outpath / 'prepared_vector.shp'

if not outpath_fields.exists():
    with rasterio.open(inpath_s2) as src:
        raster_meta = src.meta
        raster_bounds = src.bounds
    df = prepare_vector(inpath_fields, raster_meta['crs'], raster_bounds)
    outpath_fields.parent.mkdir(parents=True, exist_ok=True)
    df.to_file(outpath_fields, driver='ESRI Shapefile', encoding='cp865')
else:
    print(f'Loading from exisiting shp file... {outpath_fields.name}')
    df = gpd.read_file(str(outpath_fields), encoding='cp865')
     
print(df.info())
display(df.head(3))

Loading from exisiting shp file... prepared_vector.shp
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 0 entries
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  0 non-null      geometry
dtypes: geometry(1)
memory usage: 0.0+ bytes
None


,geometry


In [34]:
outpath_chips = outpath / 'chip_dfs.pkl'
chip_width, chip_height = 128, 128

if not outpath_chips.exists():
    with rasterio.open(inpath_s2) as src:
        raster_meta = src.meta
    chip_dfs = utils.geo.cut_chip_geometries(vector_df=df,
                                             raster_width=raster_meta['width'],
                                             raster_height=raster_meta['height'],
                                             raster_transform=raster_meta['transform'],
                                             chip_width=chip_width,
                                             chip_height=chip_height,
                                             first_n_chips = 1000)
    new_pickle(outpath_chips, chip_dfs)
else:
    chip_dfs = load_pickle(outpath_chips)
    
print('len', len(chip_dfs))


Loading from existing pickle file... chip_dfs.pkl
len 0


In [35]:
train_chip_dfs, val_chip_dfs = utils.coco.train_test_split(chip_dfs, test_size=0.2, seed=1)
coco_train = utils.coco.format_coco(train_chip_dfs, chip_width, chip_height)
coco_val = utils.coco.format_coco(val_chip_dfs, chip_width, chip_height)
new_json(outpath / r'annotations\train2016.json', coco_train)
new_json(outpath / r'annotations\val2016.json', coco_val)

Writing new json file... annotations\train2016.json
Writing new json file... annotations\val2016.json


In [36]:
# Cut to chip images and write to disk, retrieve chip image statistics
    
chip_windows = {chip_name : value['chip_window'] for chip_name, value in chip_dfs.items()}
chip_statistics = utils.img.cut_chip_images(inpath_raster=inpath_s2,
                                            outpath_chipfolder=outpath / r'images\train2016', 
                                            chip_names=chip_windows.keys(),
                                            chip_windows=chip_windows.values(), 
                                            bands=[3, 2, 1])

utils.coco.move_coco_val_images(inpath_train_folder=outpath / r'images\train2016',
                                val_chips_list=val_chip_dfs.keys())

print('len', len(chip_statistics))

0it [00:00, ?it/s]

len 0


In [25]:
statistics = {
    'nr_chips': len(chip_dfs.keys()),
    'nr_chips_train': len(train_chip_dfs),
    'nr_chips_val': len(val_chip_dfs),
    'nr_polys': sum([len(df['chip_df']) for df in chip_dfs.values()]),
    'nr_polys_train': sum([len(df['chip_df']) for df in train_chip_dfs.values()]),
    'nr_polys_val': sum([len(df['chip_df']) for df in val_chip_dfs.values()]),
#     'avg_polys_per_chip': sum([len(df['chip_df']) for df in chip_dfs.values()]) / len(chip_dfs.keys()),
#     'train_rgb_mean': list(np.asarray([df['mean'] for df in [chip_statistics[key] for key in train_chip_dfs.keys()]]).mean(axis=0)),
#     'train_rgb_std': list(np.asarray([df['std'] for df in [chip_statistics[key] for key in train_chip_dfs.keys()]]).mean(axis=0))}
}

new_json(outpath / 'statistics.json', statistics)
pprint(statistics)

Writing new json file... statistics.json
{'nr_chips': 0,
 'nr_chips_train': 0,
 'nr_chips_val': 0,
 'nr_polys': 0,
 'nr_polys_train': 0,
 'nr_polys_val': 0}


In [26]:
utils.coco.plot_coco(inpath_json=r'/Users/chetnagupta/project-arms4ai/data/output/preprocessed/annotations\val2016.json', 
                     inpath_image_folder= r'/Users/chetnagupta/project-arms4ai/data/output/preprocessed\val2016',end=2
                     )

AttributeError: 'str' object has no attribute 'name'